In [ ]:
from collections import deque
import math

import datetime

# good: q = 0
# bad: q = 1
class ValQ:
    def __init__(self, val, q):
        self.val = val
        self.q = q


class ValQT(ValQ):
    def __init__(self, val, q, t=None):
        self.val = val
        self.q = q
        if t is not None:
            self.t = t
        else:
            self.t = datetime.datetime.now()

class KlimovaBaseReturn:
    def __init__(self, Re: ValQ, Im: ValQ, ampX: ValQ, fi: ValQ):
        self.Re = Re
        self.Im = Im
        self.ampX = ampX
        self.fi = fi


class MeasKlimovaBase:
    """
    Fourier Filter with its own buffer for the stream of data.
    """

    def __init__(self, f_rated=50, f_adc=4000, frame_len: int = 40):
        self.f_rated = f_rated
        self.f_adc = f_adc
        self.frame_len = frame_len
        if frame_len % 1 != 0:
            mes = f"Frame length must be integer. Got {frame_len} instead"
            raise ValueError(mes)
        # number of points per period
        self.period_len = f_adc / f_rated
        self.period_len_error = self.period_len % 1 != 0
        self.period_len = int(round(self.period_len))
        if self.period_len_error:
            mes = f"Number of samples per period must be integer. Got {self.period_len} instead"
            raise ValueError(mes)
        # number of points per half-period
        self.half_period_len = self.period_len / 2
        self.half_period_len_error = self.half_period_len % 1 != 0
        self.half_period_len = int(round(self.half_period_len))
        if self.half_period_len_error and (self.frame_len == self.half_period_len):
            mes = f"Number of samples per half-period must be integer if half-period method is used. Got {self.half_period_len} instead"
            raise ValueError(mes)
        # check the correctness of frame length
        if self.frame_len != self.period_len and self.frame_len != self.half_period_len:
            mes = f"Frame length must be equal to {self.period_len} or {self.half_period_len}"
            raise ValueError(mes)

        k = 1  # harmonic to filter
        m = 1  # derivative order
        c = 0.5  # 0.5 for half-shifting and 1 without half-shifting
        pi = math.pi
        angle_coef = -2 * pi / self.period_len * k
        angle_shift = -2 * pi / self.period_len * (frame_len * k + c) - pi * m / 2
        # precalculating coefs
        # loops faster without self., because no class attribute call overhead
        self.cos_coefs = [2/frame_len*math.cos(angle_coef * n + angle_shift) for n in range(frame_len)]  # fmt:skip
        self.sin_coefs = [2/frame_len*math.sin(angle_coef * n + angle_shift) for n in range(frame_len)]  # fmt:skip
        # loops faster with multiplication than with division
        self.der_den_inv = self.period_len / (2 * pi)

        # ring sample buffer with length frame_len
        self.buf = deque([], maxlen=self.frame_len)
        self.der_buf = deque([], maxlen=self.frame_len)

    def fitSample(self, sample) -> KlimovaBaseReturn:
        # References reduce class member access overhead
        buf = self.buf
        der_buf = self.der_buf

        buf.append(sample)
        if len(buf) > 1:
            der_buf.append((buf[-1] - buf[-2]) * self.der_den_inv)

        if (
            len(der_buf) > 2
            and abs(der_buf[-3]) < abs(der_buf[-2])
            and abs(der_buf[-2]) > abs(der_buf[-1])
        ):
            # Сглаживание скачков производной
            der_buf[-2] = (der_buf[-3] + der_buf[-1]) * 0.5

        if len(der_buf) < self.frame_len:
            # Buffer has not been filled yet
            resRe = ValQ(0, 1)
            resIm = ValQ(0, 1)
            resAmpX = ValQ(0, 1)
            resFi = ValQ(0, 1)
        else:
            # References reduce class member access overhead
            frame_len = self.frame_len
            cos_coefs = self.cos_coefs
            sin_coefs = self.sin_coefs
            # Filtering
            Re = sum(der_buf[i] * cos_coefs[i] for i in range(frame_len))
            Im = sum(der_buf[i] * sin_coefs[i] for i in range(frame_len))
            AmpX = math.sqrt(Re**2 + Im**2)
            Fi = math.atan2(Im, Re)
            resRe = ValQ(Re, 0)
            resIm = ValQ(Im, 0)
            resAmpX = ValQ(AmpX, 0)
            resFi = ValQ(Fi, 0)
        res = KlimovaBaseReturn(resRe, resIm, resAmpX, resFi)
        return res
